In [36]:
import json
from sentence_transformers import SentenceTransformer
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import numpy as np

In [3]:
# from elasticsearch import Elasticsearch
# es_client = Elasticsearch('http://localhost:9200')

In [4]:
with open('vicRoads_faq.json','rt') as f:
    raw_documents = json.load(f)

In [5]:
raw_documents[:2]

[{'Section': 'Digital Driver Licence',
  'Question': 'How secure is a digital driver licence?\n',
  'Answer': "The security of your private information is our top priority. Your digital driver licence will be secure in the app. It can only be seen by you and whoever you choose to share it with. To protect your privacy, you decide if you want to share all - or just some - of your licence data. You control the personal information you share with licence verifiers based on why you are using it. \nThere are 3 levels of info that you can share with your digital driver licence:\nFull licence: This view shows info about your right to drive. You can share this view when you need to show police things like licence status, classes, or restrictions.\nIdentity: This shows your basic ID info — name, address, and signature.\nProof of age: This view shows that you’re over 18. It won’t show your exact age or date of birth. \nIn addition, the app is protected by the security features of your phone, inc

In [6]:
raw_documents[1]

{'Section': 'Digital Driver Licence',
 'Question': "I'm concerned about safety and privacy as someone who has experienced family violence. How can I protect my personal details?",
 'Answer': 'Customers impacted by family violence can contact us through our dedicated phone line or email:\nPhone: 1300 031 292 (business hours)\nEmail: assistance@roads.vic.gov.au\nYou can leave your contact number if the telephone line is unavailable or unattended. Our service also supports agencies acting on behalf of affected persons. \n'}

In [7]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
# checking the embedding model
embedding_model.encode("hello, how are you")

array([ 3.23615260e-02,  4.60605742e-03, -1.58994719e-02, -1.67294331e-02,
        1.28823798e-02, -6.20545680e-03, -6.78102067e-03,  4.42842953e-02,
        2.31103264e-02, -1.49674555e-02, -1.86000764e-02, -1.32769877e-02,
        1.65598094e-03,  2.24389601e-02,  2.42546736e-03, -5.31541444e-02,
       -1.32359064e-03, -3.70156728e-02, -5.00107780e-02, -1.24486675e-02,
       -2.09001992e-02,  4.09533223e-03,  6.73026033e-03,  2.53259782e-02,
        3.17776687e-02,  1.80080645e-02,  4.35382910e-02,  1.04745813e-02,
       -1.97150689e-02,  4.97864634e-02,  1.90035801e-03,  1.00770853e-02,
        6.17352687e-02,  4.44002561e-02,  1.64460425e-06,  1.55913010e-02,
       -2.94506624e-02, -3.69033329e-02,  3.42211910e-02, -3.62923965e-02,
        3.13581899e-02, -4.15903032e-02,  1.66772287e-02,  6.11308739e-02,
       -2.99137412e-03, -5.30147180e-02,  2.35420782e-02,  3.04308031e-02,
       -1.75284653e-03,  8.60268134e-04, -3.77290435e-02, -4.23870049e-02,
       -7.81421922e-03, -

In [31]:
combined_documents = []

for document in raw_documents:
    text = document['Section']+"\n"+ document['Question']+"\n"+ document['Answer']
    combined_documents.append(text)

In [10]:
combined_documents[1]

"Digital Driver Licence\nI'm concerned about safety and privacy as someone who has experienced family violence. How can I protect my personal details?\nCustomers impacted by family violence can contact us through our dedicated phone line or email:\nPhone: 1300 031 292 (business hours)\nEmail: assistance@roads.vic.gov.au\nYou can leave your contact number if the telephone line is unavailable or unattended. Our service also supports agencies acting on behalf of affected persons. \n"

In [11]:
embeddings = []

for doc in combined_documents:
    embeddings.append(embedding_model.encode(doc))

In [37]:
embeddings = np.array(embeddings).astype('float32')

In [38]:
index = faiss.IndexFlatL2(768) 

In [39]:
index.add(embeddings)

In [40]:
embeddings.shape

(157, 768)

In [64]:
query = "I am still on my P's, can I get digital licence"
query_embedding = embedding_model.encode(query)
query_embedding = np.array(query_embedding).astype('float32').reshape(1,-1)

In [65]:
k = 5  # Number of results to retrieve
distances, indices = index.search(query_embedding, k)

In [66]:
for i, idx in enumerate(indices[0]):
    if idx < len(raw_documents):  # Ensure the index is within bounds
        print(f"Result {i + 1}:")
        print(f"Section: {raw_documents[idx]['Section']}")
        print(f"Question: {raw_documents[idx]['Question']}")
        print(f"Answer: {raw_documents[idx]['Answer']}")
        print(f"Distance: {distances[0][i]}\n")
    else:
        print(f"Result {i + 1}: No corresponding FAQ found for index {idx}")

Result 1:
Section: Digital Driver Licence
Question: Once downloaded, will I be able to use my digital licence immediately at all locations where I currently present my physical licence?
Answer: In most locations, yes you will. But we ask for patience, because many establishments will still be in the process of including digital driver licence in their systems once the roll out is live. We encourage you to carry your physical licence to avoid potential issues.
Distance: 0.7248924374580383

Result 2:
Section: Digital Driver Licence
Question: How to get your digital licence?
Answer: To get your digital driver licence, you’ll need a myVicRoads account
Download the myVicRoads app via Google Play or through the App store and log in with your myVicRoads account.
Reset your password from the login screen if you need (click ‘forgotten password’)
Follow the prompts to activate two-step verification (if this isn’t already enabled).
Distance: 0.7314901351928711

Result 3:
Section: Digital Driver L